In [1]:
import srsly
data = srsly.read_jsonl('ru_time_metrics_model/corpus/train.jsonl')

In [3]:
from tqdm.autonotebook import tqdm 
train_data = []
for row in tqdm(data, total=59712):
    if row['cats']['EMPIRE'] == 1.0:
        train_data.append((row['text'],{'cats': {'EMPIRE': 1, 'NOT_EMPIRE': 0}}))
    else:
        train_data.append((row['text'],{'cats': {'EMPIRE': 0, 'NOT_EMPIRE': 1}}))

  0%|          | 0/59712 [00:00<?, ?it/s]

In [ ]:
!python -m spacy download ru_core_news_sm

In [4]:
import spacy 
nlp = spacy.load("ru_core_news_sm")
textcat = nlp.add_pipe("textcat")
textcat.add_label('EMPIRE')
textcat.add_label('NOT_EMPIRE')
#textcat.add_label('POST')

1

In [ ]:
import random
from spacy.util import minibatch, compounding
from spacy.training import Example

other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'textcat']

n_iter = 5

# Only train the textcat pipe
with nlp.disable_pipes(*other_pipes):
    optimizer = nlp.initialize()
    for i in tqdm(range(n_iter)):
        random.shuffle(train_data)
        for raw_text, cats in train_data:
            doc = nlp.make_doc(raw_text)
            example = Example.from_dict(doc, cats)
            nlp.update([example],sgd=optimizer)
            
nlp.to_disk("./empire_not_empire")  

  0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
!ls ./empire_not_empire

In [ ]:
nlp = spacy.load('./empire_not_empire')

In [ ]:
doc = nlp('В половине двенадцатого прибывает мой дорогой Оболенский; когда я спускаюсь к нему, он умывается. Будучи очень доволен проделанным путешествием, он вместе с тем несколько утомлен и страдает от сильного насморка. Завтракаем вместе с ним у меня на квартире; весь день рассказываем друг другу тысячи всяческих вещей. При свидании с Лобановым Оболенский намеревается начать разговор с увеличения жалованья Азиатскому департаменту и просить того же для канцелярии. Уговариваю его сделать это только по зрелом размышлении, обсудив все детали данного вопроса с Никоновым. Сегодня у моей кухарки именины, предоставляю ей отгул.')

In [ ]:
doc.cats